In [ ]:
import pandas as pd
import re
import hanja

In [ ]:
data = pd.read_excel("/content/drive/MyDrive/poem_crawled_data.xlsx")
print(len(data))
data.head()

17909


,Unnamed: 0,title,content,img_url
0,0,해후,\n\t\t\t\t\t\t큰 손의 터치. 시원하고 장쾌한 스크린\n두 그리움이 만나...,http://cafe186.daum.net/_c21_/pds_down_hdn?grp...
1,1,접선,\n\t\t\t\t\t\t정지된 시간대. \n정체불명 E.T 같은 자들이 접선하고 ...,http://cafe186.daum.net/_c21_/pds_down_hdn?grp...
2,2,변신,\n\t\t\t\t\t\t 나는 한때 네 그루 나무중에 하나였습니다.\n 그러나\n...,http://cafe186.daum.net/_c21_/pds_down_hdn?grp...
3,3,너는 문이다,\n\n\n너는 문(moon)이다\n\n암흑에 갇힌 者에게\n\n빛의 세계 여는 문...,http://cafe186.daum.net/_c21_/pds_down_hdn?grp...
4,4,사량도가는길,\n사량도 가는길\n\n\n \n이른 새벽\n물살을 가르며\n또 하루를 연다\n뜨거...,http://cafe186.daum.net/_c21_/pds_down_hdn?grp...


In [ ]:
#영어 들어간 content 지우기
data['drop'] = data['content'].apply(lambda x : 1 if  re.compile(r'[a-zA-Z]+').search(x) else 0)
data = data.drop(data[data['drop']==1].index)

In [ ]:
#괄호 + 괄호 안 지우기
data['content'] = data['content'].apply(lambda x : re.sub(r"\([^)]*\)","",x))

In [ ]:
#줄바꿈 바꾸기
def func_space(text):
  text = re.sub(r"\/+","/",text)
  text = re.sub(r"[ (\xa0)|(\t)]+"," ",text)
  text = re.sub(r"[(\n)]+","\n",text)
  return text

data['content'] = data['content'].apply(lambda x : func_space(x))

In [ ]:
def func_character(text):
  #한자 바꾸기
  hj = list(set(re.findall(r"([一-鿕]|[㐀-䶵]|[豈-龎])+", text)))
  hg = [hanja.translate(h,'substitution') for h in hj]
  for i in range(len(hg)):
    text = text.replace(hj[i], hg[i])
  #특수문자 지우기
  text = re.sub(r"[^가-힇0-9‘“.,!? ]+", "", text)
  return text

data['content'] = data['content'].apply(lambda x : func_character(x))

In [ ]:
data['drop'] = data['content'].apply(lambda x : 1 if len(x)<10 else 0)
data = data.drop(data[data['drop']==1].index)

In [ ]:
len(data)
data = data[['title', 'content', 'img_url']]
data.to_excel("preprocess_data.xlsx")